In [1]:
import tensorflow as tf
import numpy as np

x_data = np.array(
    [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

y_data = np.array([
    [1, 0, 0], 
    [0, 1, 0],  
    [0, 0, 1],  
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

global_step = tf.Variable(0, trainable=False, name='global_step')
X = tf.placeholder(tf.float32, name='Input')
Y = tf.placeholder(tf.float32, name='Output')

with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    b1 = tf.Variable(tf.zeros([10]), name='b1')
    L1 = tf.add(tf.matmul(X, W1), b1, name='L1')
    L1 = tf.nn.relu(L1)
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.), name='W2')
    b2 = tf.Variable(tf.zeros([3]), name='b2')
    model = tf.add(tf.matmul(L1, W2), b2, name='model')
    prediction = tf.argmax(model, 1, name='prediction')

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model), name='cost')
optimizer = tf.train.AdamOptimizer(learning_rate=0.01, name='optimizer')
train_op = optimizer.minimize(cost, global_step=global_step)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

saver = tf.train.Saver(tf.global_variables())
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

for step in range(30):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    if (step + 1) % 30 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
        tf.train.write_graph(sess.graph_def, '.', './model/FFNN.pbtxt')  
        saver.save(sess, './model/FFNN.ckpt', global_step=global_step)
        break
        
target = tf.argmax(Y, 1)
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('Accuracy: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


(30, 0.5361519)
Accuracy: 100.00


In [14]:
from tensorflow.python.tools import freeze_graph

freeze_graph.freeze_graph("model/FFNN.pbtxt", "",
                          False, "model/FFNN.ckpt-30", "layer2/prediction",
                          "", "",
                          "FFNN_frozen_graph.pb", True, "")

INFO:tensorflow:Restoring parameters from model/FFNN.ckpt-30


INFO:tensorflow:Restoring parameters from model/FFNN.ckpt-30


INFO:tensorflow:Froze 4 variables.


INFO:tensorflow:Froze 4 variables.


Converted 4 variables to const ops.


In [15]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name="")

    return graph

In [16]:
graph = load_graph('FFNN_frozen_graph.pb')

for op in graph.get_operations():
    print(op.name)

Input
layer1/W1
layer1/W1/read
layer1/b1
layer1/b1/read
layer1/MatMul
layer1/L1
layer1/Relu
layer2/W2
layer2/W2/read
layer2/b2
layer2/b2/read
layer2/MatMul
layer2/model
layer2/prediction/dimension
layer2/prediction


In [20]:
import tfcoreml

mlmodel = tfcoreml.convert(
        tf_model_path = 'FFNN_frozen_graph.pb',
        mlmodel_path = 'FFNN.mlmodel',
        output_feature_names = ['layer2/prediction:0'],
        input_name_shape_dict = {'Input:0': [1, 2]})

Shapes not found for 7 tensors. Executing graph to determine shapes. 
1/16: Analysing op name: layer2/prediction/dimension ( type:  Const )
2/16: Analysing op name: layer2/b2 ( type:  Const )
3/16: Analysing op name: layer2/b2/read ( type:  Identity )
4/16: Analysing op name: layer2/W2 ( type:  Const )
5/16: Analysing op name: layer2/W2/read ( type:  Identity )
6/16: Analysing op name: layer1/b1 ( type:  Const )
7/16: Analysing op name: layer1/b1/read ( type:  Identity )
8/16: Analysing op name: layer1/W1 ( type:  Const )
9/16: Analysing op name: layer1/W1/read ( type:  Identity )
10/16: Analysing op name: Input ( type:  Placeholder )
Skipping name of placeholder
11/16: Analysing op name: layer1/MatMul ( type:  MatMul )
12/16: Analysing op name: layer1/L1 ( type:  Add )
13/16: Analysing op name: layer1/Relu ( type:  Relu )
14/16: Analysing op name: layer2/MatMul ( type:  MatMul )
15/16: Analysing op name: layer2/model ( type:  Add )
16/16: Analysing op name: layer2/prediction ( type:  

In [26]:
import tensorflow as tf

graph_def_file = "FFNN_frozen_graph.pb"
input_arrays = ["Input"]
output_arrays = ["layer2/prediction"]

converter = tf.contrib.lite.TocoConverter.from_frozen_graph(
  graph_def_file, input_arrays, output_arrays)
tflite_model = converter.convert()
open("FFNN.tflite", "wb").write(tflite_model)

AttributeError: 'module' object has no attribute 'TocoConverter'